In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch

from netam import noof, shmoof
from epam.torch_common import pick_device

In [2]:
full_shmoof_df = pd.read_csv("/Users/matsen/data/shmoof_edges_11-Jan-2023_NoNode0_iqtree_K80+R_masked.csv", index_col=0).reset_index(drop=True)

# only keep rows where parent is different than child
full_shmoof_df = full_shmoof_df[full_shmoof_df["parent"] != full_shmoof_df["child"]]

# full_shmoof_df = full_shmoof_df.sample(5000)

# Hold out sample 326713
val_df = full_shmoof_df[full_shmoof_df["sample_id"] == 326713]
train_df = full_shmoof_df.drop(val_df.index)

full_shmoof_df.sample_id.value_counts()

sample_id
326651    22424
326713    13186
327059     4686
316188     3067
326797     3028
326780     1573
326737      442
326650      324
326907      286
Name: count, dtype: int64

In [3]:
kmer_length = 5
max_length = 500

train_dataset = shmoof.SHMoofDataset(train_df, kmer_length=kmer_length, max_length=max_length)
val_dataset = shmoof.SHMoofDataset(val_df, kmer_length=kmer_length, max_length=max_length)

print(f"we have {len(train_dataset)} training examples and {len(val_dataset)} validation examples")

we have 35830 training examples and 13186 validation examples


In [4]:
model = noof.NoofModel(train_dataset, embedding_dim=2, nhead=2, dim_feedforward=512, layer_count=3, dropout=0.1)

device = pick_device()
train_dataset.to(device)
val_dataset.to(device)
model.to(device)

burrito = noof.NoofBurrito(train_dataset, val_dataset, model, learning_rate=0.1, l2_regularization_coeff=1e-6)
print("starting training...")
losses = burrito.train(epochs=10)

Using Metal Performance Shaders
starting training...
Epoch [1/10]	 Loss: 17.779089	 Val Loss: 20.12041
Epoch [2/10]	 Loss: 17.776786	 Val Loss: 20.119459
Epoch [3/10]	 Loss: 17.776683	 Val Loss: 20.119612
Epoch [4/10]	 Loss: 17.776736	 Val Loss: 20.119473
Epoch [5/10]	 Loss: 17.776671	 Val Loss: 20.119497
Epoch [6/10]	 Loss: 17.776854	 Val Loss: 20.119454
Epoch [7/10]	 Loss: 17.776714	 Val Loss: 20.119643
Epoch [8/10]	 Loss: 17.776706	 Val Loss: 20.119541
Epoch [9/10]	 Loss: 17.776675	 Val Loss: 20.120012
Epoch [10/10]	 Loss: 17.776774	 Val Loss: 20.11947


In [5]:
train_dataset[0][0]

tensor([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,  415,  636,  494,  952,  734,
         888,  479,  891,  492,  944,  701,  756,  976,  830,  246,  981,  852,
         334,  309,  211,  842,  293,  147,  588,  303,  187,  748,  944,  701,
         754,  968,  800,  126,  504,  991,  891,  491,  938,  678,  664,  607,
         379,  489,  932,  654,  566,  215,  859,  362,  421,  659,  586,  294,
         150,  598,  342,  341,  339,  331,  299,  169,  673,  643,  523,   43,
         169,  674,  648,  543,  123,  489,  931,  652,  559,  187,  745,  932,
         656,  575,  251, 1003,  937,  6

In [6]:
untrained_model = noof.NoofModel(train_dataset, embedding_dim=2, nhead=2, dim_feedforward=512, layer_count=3, dropout=0.5)


#untrained_model(torch.tensor(train_dataset[0][0]).unsqueeze(0).to("cpu"))
untrained_model(torch.tensor(train_dataset[0:5][0]).to("cpu"))[:,100:105]

/var/folders/16/pkph6n2962g3lf2wpwwt0wwr0000gn/T/ipykernel_67957/838397735.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  untrained_model(torch.tensor(train_dataset[0:5][0]).to("cpu"))[:,100:105]


tensor([[1.0246, 1.0246, 0.9760, 1.0246, 1.0246],
        [1.0246, 1.0246, 0.9760, 0.9760, 1.0246],
        [1.0246, 0.9760, 0.9760, 1.0246, 0.9760],
        [1.0246, 0.9760, 0.9760, 0.9760, 0.9760],
        [0.9760, 0.9760, 0.9760, 1.0246, 1.0246]], grad_fn=<SliceBackward0>)

In [7]:
model(torch.tensor(train_dataset[0:5][0]))[:,100:105]

/var/folders/16/pkph6n2962g3lf2wpwwt0wwr0000gn/T/ipykernel_67957/3090242575.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  model(torch.tensor(train_dataset[0:5][0]))[:,100:105]


tensor([[1.0236, 1.0236, 1.0236, 1.0236, 1.0236],
        [1.0236, 1.0236, 1.0236, 1.0236, 1.0236],
        [1.0236, 1.0236, 1.0236, 1.0236, 1.0236],
        [1.0236, 1.0236, 1.0236, 1.0236, 1.0236],
        [1.0236, 1.0236, 1.0236, 1.0236, 1.0236]], device='mps:0',
       grad_fn=<SliceBackward0>)

In [8]:
torch.save(model, "_ignore/noof_model.pt")

In [9]:
xmodel = torch.load("_ignore/noof_model.pt")
xmodel.eval()

xmodel(val_dataset.encoded_parents)[0]

RuntimeError: The size of tensor a (13186) must match the size of tensor b (5000) at non-singleton dimension 0

In [ ]:
val_dataset.masks[0]

tensor([False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True, 